In [93]:
from pathlib import Path

import pandas as pd

cwd = !pwd
cwd = Path(str(cwd[0]))
file_name = "murphys_clan.txt"
raw_file_path = cwd.parent / "raw_export" / file_name

def get_convo_as_list(raw_file_path):
    with open(raw_file_path, "r") as f:
        lines = f.read().splitlines()
    return lines

def get_raw_series(raw_text):
    raw_series = pd.Series(raw_text)
    return raw_series

def process_dataframe(raw_series):
    df = pd.DataFrame()
    df[["date", "other"]] = raw_series.str.split("-", n=1, expand=True)
    df[["person", "message"]] = df["other"].str.split(":", n=1, expand=True)
    df.drop("other", inplace=True, axis=1)
    return df

## Look see of initial read in method

In [94]:
raw_text = get_convo_as_list(raw_file_path)
raw_series = get_raw_series(raw_text)
df = process_dataframe(raw_series.copy())

In [95]:
is_nan = df.isnull()
is_nan = pd.DataFrame(df.index[is_nan['person']], columns=["null_rows"])
is_nan['shift'] = is_nan['null_rows'].shift(-1)
is_nan['shift'].astype('int64', errors='ignore')

0        534.0
1        535.0
2        536.0
3        537.0
4        538.0
        ...   
765    23565.0
766    23616.0
767    23617.0
768    23637.0
769        NaN
Name: shift, Length: 770, dtype: float64

In [97]:
is_nan['is_sequence'] = is_nan['shift'] == is_nan['null_rows'] +1
s = is_nan['is_sequence']
is_nan['sequence'] = s[s].groupby((~s).cumsum()).ngroup()
df.iloc[23533:23537]

,date,person,message
23533,Thanks Tim x,None,None
23534,,None,None
23535,https://forms.office.com/Pages/ResponsePage.as...,None,None
23536,"04/10/2021, 13:46",Mum,Not sure whether either of you would like to ...
